In [1]:
import os

In [2]:
%pwd

'/Users/vikkyfury/Desktop/pr/program/LangWiz-AI/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/vikkyfury/Desktop/pr/program/LangWiz-AI'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path
    

In [ ]:
from LangWizAI.constants import *
from LangWizAI.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir = self.config.root_dir,
            data_path = self.config.data_path,
            tokenizer_name = config.tokenizer_name
        )
        return data_transformation_config

In [ ]:
import os
from LangWizAI import logger
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk

In [ ]:
class DataTransformation:
    def __init__(self,config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pre_trained(config.tokenizer_name)
        
        self.model = AutoModelForSeq2SeqLM.from_pretrained(config.tokenizer_name)
    
    def translate(self,example):
        inputs = example['inputs']
        targets = example['targets']
        if not isinstance(inputs,(str,list)):
            inputs = str(inputs)
        if not isinstance(targets,(str,list)):
            targets = str(targets)
        input_encodings = self.tokenizer(inputs,max_length = 128, truncation = True, return_tensors = 'pt')
        with self.tokenizer.as_target_tokenizer():
            target_encodings= self.tokenizer(targets, max_length = 128, truncation = True, return_tensors = 'pt')
        
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask' : input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']       
        }
    
    def convert(self):
        data = load_from_disk(self.config.data_path)
        data_pt = data.map(self.translate,batched =True)
        data_pt.save_to_disk(os.path.join(self.config.root_dir,'fr-en'))

In [ ]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.convert()
